In [4]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val username = "Johann-27.1"

val monthlyReportFrom = YearMonthTO(2019, 1)
val monthlyReportTo = YearMonthTO(2021, 3)
val monthlyReportForecastUntil = YearMonthTO(2021, 9)

val yearlyReportFrom = 2019
val yearlyReportTo = 2020
val yearlyReportForecastUntil = 2022

val fundName = "Work Income"
val reportViewName = "Work income report"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/work-report-data-configuration.yaml"

val client = FundsClient()

In [5]:
val user = client.ensureUserExists(username)
user

UserTO(id=b3d83549-abdb-409b-8d5d-d6562317209f, username=Johann-27.1)

In [6]:
val reportView = client.createReportView(user, reportViewName, fundName, client.fromYaml(File(REPORT_DATA_CONFIGURATION_YAML_FILE), "dataConfiguration"))
reportView

ReportViewTO(id=0cc6f12d-df3e-48c2-8832-47dd0df48848, name=Work income report, fundId=b92271c2-92a8-433d-984a-c06986806dd5, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[income, work_taxes]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]), forecast=ForecastReportFeatureTO(inputBuckets=1))))

In [7]:
val monthlyReportData = client.getMonthlyReportViewData(user, reportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [8]:
val yearlyReportData = client.getYearlyReportViewData(user, reportViewName, yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil)


In [9]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupedNetData(reportData: ReportDataTO, plotTitle: String, incomeGroupName: String, taxesGroup: String): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "income" to reportData.data.map { bucket ->
            bucket.groupedNet!!.firstOrNull { it.group == incomeGroupName }?.net ?: BigDecimal.ZERO
        },
        "taxes" to reportData.data.map { bucket ->
            bucket.groupedNet!!.firstOrNull { it.group == taxesGroup }?.net ?: BigDecimal.ZERO
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedNet!!.mapNotNull { it.net }.fold(BigDecimal.ZERO) { acc, value -> acc + value }
        }
    )

    return dataFrame
        .plot {
            line {
                val values = dataFrame.get("net").values.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map { it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds() }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("net")
                color = Color.YELLOW
            }
            line {
                y("taxes")
                color = Color.RED
            }
            line {
                y("income")
                color = Color.GREEN
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }

}

In [10]:
plotGroupedNetData(yearlyReportData, "Net income per year", "income", "taxes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="GXoSwo"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Net income per year"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"income":[107567.0,138677.75,123122.375,123122.375],
"taxes":[0.0,0.0,0.0,0.0],
"net":[107567.0,138677.75,123122.375,123122.375]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[0.0,138677.75]
},
"sampling":"none",
"x":1.5935616E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"net"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"taxes"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"income"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("GXoSwo");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 
 
 -5,000 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 15,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 25,000 
 
 
 
 
 
 
 30,000 
 
 
 
 
 
 
 35,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 45,000 
 
 
 
 
 
 
 50,000 
 
 
 
 
 
 
 55,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 65,000 
 
 
 
 
 
 
 70,000 
 
 
 
 
 
 
 75,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 85,000 
 
 
 
 
 
 
 90,000 
 
 
 
 
 
 
 95,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 105,000 
 
 
 
 
 
 
 110,000 
 
 
 
 
 
 
 115,000 
 
 
 
 
 
 
 120,000 
 
 
 
 
 
 
 125,000 
 
 
 
 
 
 
 130,000 
 
 
 
 
 
 
 135,000 
 
 
 
 
 
 
 140,000 
 
 
 
 
 
 
 145,000 
 
 
 
 
 
 
 
 
 Net incom

In [11]:
plotGroupedNetData(monthlyReportData, "Income per month", "income", "taxes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="TGUROq"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Income per month"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"income":[6950.0,7006.0,14784.0,7037.0,8157.0,8227.0,8184.0,8344.0,8007.0,10203.0,10287.0,10381.0,10176.0,10237.0,10229.0,19367.0,11411.0,10040.0,10082.0,10489.0,10110.0,10124.0,11694.0,14718.75,13596.0,13500.0,13595.0,12393.89583333333,11812.80381944444,11846.28747106481,11996.81142698688,12156.37904590246,12295.32729972766],
"taxes":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-500.0,-41.66666666666667,-45.13888888888889,-48.90046296296296,-52.97550154320988,-57.3901266718107,-62.17263722779492],
"net":[6950.0,7006.0,14784.0,7037.0,8157.0,8227.0,8184.0,8344.0,8007.0,10203.0,10287.0,10381.0,10176.0,10237.0,10229.0,19367.0,11411.0,10040.0,10082.0,10489.0,10110.0,10124.0,11694.0,14718.75,13596.0,13500.0,13095.0,12352.229166666662,11767.664930555551,11797.387008101847,11943.835925443671,12098.98891923065,12233.154662499865]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12